<a href="https://colab.research.google.com/github/alondonoco/SenalesSistemas/blob/main/ProyectoFinal/Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instalación de librerías**

In [1]:
#instalación de librerías
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.5 MB/s eta 0:00:00


In [2]:
!pip install streamlit numpy scipy matplotlib yt-dlp pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 62.5 MB/s eta 0:00:00


##Crear carpeta pages para trabajar Multiapp en Streamlit

In [3]:
!mkdir pages

# **Página principal**

In [4]:
%%writefile 0_👋_Hello.py

import streamlit as st

st.set_page_config(
    page_title="Proyecto de SyS",
    page_icon="👋",
    layout="wide"
)

st.title("Proyecto Final SyS")

st.sidebar.success("Selecciona una página para explorar.")

st.markdown("""
Bienvenido al desarrollo del Proyecto Final de SyS

---

**Información de los Estudiantes:**

*   **Nombre:** Alejandro Londoño Correa
*   **Cédula:** 1055750510
---
*   **Nombre:** Kevin Loaiza Patiño
*   **Cédula:** 1054478784

"""
)

Writing 0_👋_Hello.py


# **Páginas**

Cada pagina se debe enviar al directorio \pages

#**PUNTO1️⃣**

In [5]:
%%writefile 1_PUNTO_1️⃣.py



Writing 1_PUNTO_1️⃣.py


In [6]:
!mv 1_PUNTO_1️⃣.py pages/

#**PUNTO2️⃣**

In [7]:
%%writefile 2_PUNTO_2️⃣.py



Writing 2_PUNTO_2️⃣.py


In [8]:
!mv 2_PUNTO_2️⃣.py pages/

# **Inicialización del Dashboard a partir de túnel local**

1. **Reemplazar nombre de archivo**: Reemplaza el nombre del archivo como se indica en el comentario de la linea 6 de la celda de codigo

2. **Accede al enlace provisional**: Una vez que la aplicación esté corriendo, LocalTunnel generará un enlace temporal. Haz clic o copia ese enlace para acceder a tu aplicación en el navegador (cada vez que corras la celda, el link podrá ser diferente).

**Nota:**
Para finalizar la ejecución del Dashboard ejecuta la ultima celda de codigo y sigue las instrucciones.

In [9]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

#Ejecutar Streamlit
!streamlit run 0_👋_Hello.py &>/content/logs.txt & #Cambiar 0_👋_Hello.py por el nombre de tu archivo principal

#Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

#Leer la URL pública generada por Cloudflare
import time
time.sleep(5)  # Esperar que se genere la URL

import re
found_context = False  # Indicador para saber si estamos en la sección correcta

with open('/content/cloudflared.log') as f:
    for line in f:
        #Detecta el inicio del contexto que nos interesa
        if "Your quick Tunnel has been created" in line:
            found_context = True

        #Busca una URL si ya se encontró el contexto relevante
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)  #Extrae la URL encontrada
                print(f'Tu aplicación está disponible en: {url}')
                break  #Termina el bucle después de encontrar la URL

--2025-07-10 22:38:23--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.7.0/cloudflared-linux-amd64 [following]
--2025-07-10 22:38:23--  https://github.com/cloudflare/cloudflared/releases/download/2025.7.0/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/37d2bad8-a2ed-4b93-8139-cbb15162d81d?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-07-10T23%3A28%3A43Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-07-10T2